In [2]:
print(9)

9


In [3]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


In [4]:
# Load embedding model (you can choose another model)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Collect all quotes
quotes = []
file_dir = './processed/'

for filename in os.listdir(file_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(file_dir, filename), 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if ':' in line:
                    author, quote = line.split(':', 1)
                    quotes.append(quote.strip())

# Convert quotes to embeddings
embeddings = model.encode(quotes, convert_to_numpy=True)

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Optional: save the index
faiss.write_index(index, 'quotes_faiss.index')

# Optional: save quotes for lookup later
with open('quotes_lookup.txt', 'w', encoding='utf-8') as f:
    for quote in quotes:
        f.write(quote + '\n')

print(f"Indexed {len(quotes)} quotes.")

Indexed 1786 quotes.


In [6]:
# Load index
index = faiss.read_index('quotes_faiss.index')

# Load quotes
with open('quotes_lookup.txt', 'r', encoding='utf-8') as f:
    quotes = [line.strip() for line in f.readlines()]

# Query
query = "I'm feeling lost."
query_vector = model.encode([query], convert_to_numpy=True)

# Search top 5
D, I = index.search(query_vector, 5)

# Display results
for idx in I[0]:
    print(quotes[idx])




We lost because we told ourselves we lost.
Loss is nothing else but change,and change is Natures delight.
Fear not for the future, weep not for the past.
Accept challenges, so that you may feel the exhilaration of victory.
When you lose, don't lose the lesson.
